In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Prepare Dataset

In [ ]:
!huggingface-cli login # need code for writing

In [ ]:
from datasets import load_dataset

In [ ]:
pip install requests jsonlines pandas

In [ ]:
import requests
import jsonlines
import pandas as pd

In [ ]:
# already done => dont' have to
# CSV 파일 로드
df = pd.read_csv('dataset.csv_path')
output_directory = 'dataset_output_path'
metadata = []

for index, row in df.iterrows():
    # 이미지 URL 가져오기
    image_url = row['image']
    caption = row['prompt ']

    try:
        # 이미지 다운로드
        response = requests.get(image_url, stream=True)
        response.raise_for_status()

        # 파일명 생성
        file_name = f'{index + 1:04d}.png'

        # 파일 경로 생성
        file_path = os.path.join(output_directory, file_name)

        # 이미지 저장
        with open(file_path, 'wb') as file:
            file.write(response.content)
        # print(f'이미지 {file_path} 저장 완료')

        # 메타데이터 생성
        metadata.append({
            'file_name': file_name,
            'text': caption
        })

    except Exception as e:
        print(f'이미지 {file_name} 저장 실패: {str(e)}')

metadata_file_path = os.path.join(output_directory, 'metadata.jsonl')

# 메타데이터 파일 작성
with jsonlines.open(metadata_file_path, 'w') as writer:
    writer.write_all(metadata)

# print(f'메타데이터 파일 {metadata_file_path} 작성 완료')


In [ ]:
# JSONL 파일 읽기
with jsonlines.open('dataset_jsonl_output_path', 'r') as reader:
    for item in reader:
        print(item)

In [ ]:
dataset = load_dataset("imagefolder", data_dir="dataset_output_path", split="train")
dataset[0]["text"]

In [ ]:
dataset

In [ ]:
dataset.push_to_hub("Hugging Face Datasets Name") #hugging face에서 creat dataset

# LoRA for text_to _image
https://github.com/huggingface/diffusers/blob/main/examples/text_to_image/README.md
https://github.com/huggingface/diffusers/blob/main/examples/text_to_image/train_text_to_image.py

In [ ]:
import os

# GitHub 저장소 복제
!git clone https://github.com/huggingface/diffusers

# 디렉토리 변경
os.chdir('diffusers_dir')

# 필요한 패키지 설치
!pip install .

In [ ]:
os.chdir('./examples/text_to_image')

In [ ]:
pip install -r requirements.txt

In [ ]:
!accelerate config default

In [ ]:
pip install --upgrade tensorrt

# Train

In [ ]:
!accelerate launch --mixed_precision="fp16" train_text_to_image_lora.py \
  --pretrained_model_name_or_path="CompVis/stable-diffusion-v1-4" \
  --dataset_name="Hugging Face Datasets Name" --caption_column="text" \
  --resolution=512 --random_flip \
  --train_batch_size=1 \
  --num_train_epochs=100 --checkpointing_steps=5000 \
  --learning_rate=1e-04 --lr_scheduler="constant" --lr_warmup_steps=0 \
  --seed=42 \
  --output_dir="model_output_dir" \
  --validation_prompt="A book cover, Title is The Deep Sky, genre is Sci-Fi, about a mission into deep space that begins with a lethal explosion that leaves the survivors questioning the loyalty of the crew."


# Generate
    https://github.com/huggingface/diffusers/tree/main/examples/text_to_image
    ratio : https://huggingface.co/blog/stable_diffusion

In [ ]:
os.chdir('output_image_dir')

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_path = "model_output_dir"
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipe.unet.load_attn_procs('model_path')
pipe.to("cuda")

In [ ]:
prompt = "book cover titled nuclear war. jenre is sci-fi. description is that the mother became a warrior to protect her kids during the nuclear war against aliens, "
image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5, height=512, width=768).images[0]
image.save("bookCover_07.png")



In [ ]:
!pwd